In [ ]:
!pip install -q datasets evaluate nltk sacremoses peft

In [1]:
import wandb
# Disable wandb logging
wandb.init(mode="disabled")

In [2]:
import pandas as pd
import numpy as np
import torch
from datasets import Dataset
from tqdm import tqdm
from nltk.translate.bleu_score import sentence_bleu
import nltk
from sklearn.model_selection import train_test_split
from transformers import (M2M100ForConditionalGeneration,
                          M2M100Tokenizer,
                          Seq2SeqTrainingArguments, Seq2SeqTrainer,
                          Trainer,
                          TrainingArguments,
                          pipeline,
                          EarlyStoppingCallback)
from peft import get_peft_model, LoraConfig, TaskType

2024-08-03 20:45:56.497131: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-03 20:45:56.497193: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-03 20:45:56.498636: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Download the necessary NLTK data
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
# Define compute_metrics function for BLEU score
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Compute BLEU scores
    bleu_scores = []
    for pred, label in zip(decoded_preds, decoded_labels):
        reference = nltk.word_tokenize(label)
        candidate = nltk.word_tokenize(pred)
        bleu_score = sentence_bleu([reference], candidate, weights=(0.5, 0.5))  # 2-gram BLEU
        bleu_scores.append(bleu_score)
    
    return {"bleu": sum(bleu_scores) / len(bleu_scores)}

## Prepair Model and Tokenizer

In [5]:
# Load the pre-trained model and tokenizer
model_name = "facebook/m2m100_1.2B"
model = M2M100ForConditionalGeneration.from_pretrained(model_name)
tokenizer = M2M100Tokenizer.from_pretrained(model_name)

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [6]:
# Set the source and target languages
tokenizer.src_lang = "th"  # Northern Thai (you may need to check the exact code)
tokenizer.tgt_lang = "th"   # Central Thai

In [7]:
# Define LoRA Config
peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules="all-linear", #["q_proj", "v_proj"]
)

# Apply LoRA to the model
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 11,796,480 || all params: 1,251,266,560 || trainable%: 0.9428


In [8]:
# Tokenize the dataset
def tokenize_function(examples):
    inputs = tokenizer(examples["Northern"], truncation=True, padding="max_length", max_length=128)
    with tokenizer.as_target_tokenizer():
        targets = tokenizer(examples["Central"], truncation=True, padding="max_length", max_length=128)
    return {
        "input_ids": inputs.input_ids,
        "attention_mask": inputs.attention_mask,
        "labels": targets.input_ids,
    }

## Load Train Data

In [9]:
# Load the dataset
df = pd.read_csv('/kaggle/input/north-translation/train.csv')
dataset = Dataset.from_pandas(df)

# Split the dataset
train_test = dataset.train_test_split(test_size=0.1)

In [10]:
tokenized_train = train_test['train'].map(tokenize_function, batched=True)
tokenized_test = train_test['test'].map(tokenize_function, batched=True)

Map:   0%|          | 0/1440 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4016: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/160 [00:00<?, ? examples/s]

## Training

In [11]:
# Set up training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy= "epoch", #"steps","epoch"
#     eval_steps = 20,
    save_strategy = "epoch", #"steps",
#     save_steps = 20,
    logging_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=30,
    predict_with_generate=True,
#     fp16=torch.cuda.is_available(),  # Enable mixed precision training if available
    metric_for_best_model="bleu",
    load_best_model_at_end=True,
)

# Define the EarlyStoppingCallback
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3, # Number of evaluations with no improvement to wait before stopping
    early_stopping_threshold=0.01 # Minimum change to qualify as an improvement
)

# Initialize the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping],
)

In [21]:
# Train the model
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Bleu
1,5.224300,5.284739,0.629929
2,5.212700,5.278725,0.634075
3,5.198200,5.275057,0.639104
4,5.188700,5.269382,0.646211


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=1440, training_loss=5.205973222520616, metrics={'train_runtime': 3506.7219, 'train_samples_per_second': 12.319, 'train_steps_per_second': 3.08, 'total_flos': 4954874928168960.0, 'train_loss': 5.205973222520616, 'epoch': 4.0})

In [ ]:
# Save the fine-tuned model
peft_model_id = "M2M100_1.2B_peft"
trainer.model.save_pretrained(peft_model_id)
tokenizer.save_pretrained(peft_model_id)

## Inference

In [ ]:
model_name = "facebook/m2m100_1.2B"
model_path = 'kaggle/working/M2M100_1.2B_peft'

def load_peft_model(model_path):
    # Load the base model
    base_model = M2M100ForConditionalGeneration.from_pretrained(model_name)

    # Load the PEFT configuration
    peft_config = PeftConfig.from_pretrained(model_path)

    # Load the PEFT model
    model = PeftModel.from_pretrained(base_model, model_path)

    return model

# Load the saved model
model = load_peft_model(model_path)
tokenizer = M2M100Tokenizer.from_pretrained(model_path)

In [16]:
# Move model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
model.eval()

# Batch inference function
def batch_translate(texts, batch_size=2):
    all_translations = []
    
    for i in tqdm(range(0, len(texts), batch_size)):
        batch_texts = texts[i:i+batch_size]
        
        # Tokenize the input
        inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
        
        # Generate translation
        with torch.no_grad():
            outputs = model.generate(**inputs, forced_bos_token_id=tokenizer.lang_code_to_id["th"])
        
        # Decode the output
        translations = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        all_translations.extend(translations)
    
    return all_translations

In [17]:
testdf=pd.read_csv('/kaggle/input/north-translation/test (2).csv')
testdf

,ID,Northern
0,0,วัน นี้ มี ประ ชุม หัว หน้า บอก ว่า ต้อง เข้า ...
1,1,ยาย พิศ เปิ้น ตึง เป๋น คน ดี นุ่ง ขาว ห่ม ขาว ...
2,2,แฟชั่น แบบ ไทย ไทย เฮา ปอ นาง แบบ เตียว ออก มา...
3,3,น้อง ว่า จะ ไป ฮับ จ้าง ขาย คัว ตี้ กาด แลง จะ...
4,4,บ่ ว่า แม่ ญิง บ่ ว่า ป้อ จาย บ่า เดี่ยว นี้ ข...
...,...,...
395,395,ย้าง แอ่ว ป่า แอ่ว ดอย ก่อน เปี่ยน มา เข้า วัด...
396,396,เอา สะ ตัง ก้า พวง มา ลัย ไป นึ่ง ร้อย บาท เอา...
397,397,จะ ไป ไป สฺว่าย กิ่ง บ่า มุด แฮง กะ เดียว หน่ว...
398,398,อิ ปี้ ก้า ซื้อ คัว แล้ว นี่ ได้ ปล๋า ดุก ปิ้ง...


In [18]:
northern_thai_texts = list(testdf['Northern'])

# Perform batch translation
translated_texts = batch_translate(northern_thai_texts,batch_size=10)

100%|██████████| 40/40 [02:42<00:00,  4.05s/it]


In [19]:
testdf['Central'] = translated_texts
testdf

,ID,Northern,Central
0,0,วัน นี้ มี ประ ชุม หัว หน้า บอก ว่า ต้อง เข้า ...,วัน นี้ มี ประ ชุม หัว หน้า บอก ว่า ต้อง เข้า ...
1,1,ยาย พิศ เปิ้น ตึง เป๋น คน ดี นุ่ง ขาว ห่ม ขาว ...,ยาย พิศ เขา ทั้ง เป็น คน ดี นุ่ง ขาว ห่ม ขาว ข...
2,2,แฟชั่น แบบ ไทย ไทย เฮา ปอ นาง แบบ เตียว ออก มา...,แฟชั่น แบบ ไทย ไทย เรา พอ นาง แบบ เดิน ออก มา ...
3,3,น้อง ว่า จะ ไป ฮับ จ้าง ขาย คัว ตี้ กาด แลง จะ...,น้อง ว่า จะ ไป รับ จ้าง ขาย ของ ที่ ตลาด เย็น ...
4,4,บ่ ว่า แม่ ญิง บ่ ว่า ป้อ จาย บ่า เดี่ยว นี้ ข...,ไม่ ว่า ผู้ หญิง ไม่ ว่า พ่อ ชาย ตอน นี้ ขี่ ร...
...,...,...,...
395,395,ย้าง แอ่ว ป่า แอ่ว ดอย ก่อน เปี่ยน มา เข้า วัด...,ย้าง เที่ยว ป่า เที่ยว ดอย ก่อน เปลี่ยน มา เข้...
396,396,เอา สะ ตัง ก้า พวง มา ลัย ไป นึ่ง ร้อย บาท เอา...,เอา เงิน แต่ พวง มา ลัย ไป หนึ่ง ร้อย บาท เอา ...
397,397,จะ ไป ไป สฺว่าย กิ่ง บ่า มุด แฮง กะ เดียว หน่ว...,อย่า ไป หา กิ่ง มะ มุด แรง ก็ เดี๋ยว นู่น ที่ ...
398,398,อิ ปี้ ก้า ซื้อ คัว แล้ว นี่ ได้ ปล๋า ดุก ปิ้ง...,พี่ แต่ ซื้อ ของ แล้ว นี่ ได้ ปลา ดุก ปิ้ง ตรง...


In [20]:
sub = testdf[['ID','Central']]
sub.to_csv('submission_M2M100_1.2_PEFT.csv',index=False)